In [1]:
import pandas as pd

In [ ]:
DATA_PATH = "../data/LIAR/"

train_df = pd.read_csv(DATA_PATH + "train.tsv", sep="\t")
val_df = pd.read_csv(DATA_PATH + "valid.tsv", sep="\t")
test_df = pd.read_csv(DATA_PATH + "test.tsv", sep="\t")

In [ ]:
columns = [
  "id",
  "label",
  "statement",
  "subject",
  "speaker",
  "speaker_job_title",
  "state_info",
  "party_affiliation",
  "barely_true_counts",
  "false_counts",
  "half_true_counts",
  "mostly_true_counts",
  "pants_on_fire_counts",
  "context",
]

train_df.columns = columns
val_df.columns = columns
test_df.columns = columns

,id,label,statement,subject,speaker,speaker_job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [8]:
with open("statements.txt", "w") as f:
  for statement in test_df["statement"]:
    f.write(statement + "\n")

In [ ]:
df = pd.read_csv("../data/politifact_statements.csv")
print(f"Original size: {len(df)}")


def remove_spanish(txt: str) -> bool:
  spanish_chars = re.findall(r"[áéíóúñü¿¡]", txt.lower())
  return len(spanish_chars) == 0


def clean_text(txt: str) -> str:
  txt = txt.lower()
  txt = re.sub(r"[^a-z0-9\s.,?!]", " ", txt)
  txt = re.sub(r"\s+", " ", txt).strip()
  return txt


df["statement"] = (
  df["statement"]
  .str.strip('"')
  .str.replace("&", " and ")
  .str.replace(r"\\u\d{3,4}", "", regex=True)
)

df = df[df["statement"].apply(remove_spanish)]
df["statement"] = df["statement"].apply(clean_text)

print(f"Cleaned size: {len(df)}")

Original size: 25999
Cleaned size: 25699


/tmp/ipykernel_42533/202920522.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["statement"] = df_clean["statement"].apply(clean_text)


In [34]:
df_clean["statement"][:50]

0     this thanksgiving, turkey prices are at their ...
1     the trump administration removed nursing from ...
2     i have just gotten the highest poll numbers of...
3     there s about 1,400 criminal illegal aliens th...
4     west virginia is the only state losing populat...
5     a pro donald trump montana town planned a pedo...
6     we re not cutting science. we re not cutting r...
7     housing is expensive because we flooded the co...
8     stock prices for health insurance companies ar...
9     video shows president donald trump patting pre...
10    evidence suggests that 1 in 25 women who consu...
11    my tariffs ... are helping to slash the defici...
12    walmart s 2025 package of thanksgiving dinner ...
13    doge halts yearly payment of 2.5 million to ba...
14    north carolina teachers are already the lowest...
15    rfk jr. flees the scene after novo nordisk exe...
16    new york city mayor elect zohran mamdani is br...
17                      voting in california is 

In [ ]:
import re

# [^\w\.,?!\s\)\('-:\"\]\[\$;%#=&@><£]

with open("statements_weird.txt", "w") as f:
  for statement in df_clean["statement"]:
    if re.findall(r"[^\w\.,?!\s\)\('-:\"\]\[\$;%#=&@><£]", statement):
      f.write(statement + "\n")

In [89]:
# train_df = pd.read_csv("data/fake_news_dataset_2.csv")
# test_df = pd.read_csv("data/fake_news_test_dataset_2.csv")
# val_df = pd.read_csv("data/fake_news_validation_dataset_2.csv")
# df = pd.concat([train_df, test_df, val_df], ignore_index=True)
# df.drop("Unnamed: 0", axis=1, inplace=True)
# # df.drop_duplicates(subset=["title"], keep="last", inplace=True)
# df = df.sample(frac=1)
# df.reset_index(inplace=True)
# len(df)

In [4]:
true = pd.read_csv("../data/True.csv")
fake = pd.read_csv("../data/Fake.csv")
true["label"] = 1
fake["label"] = 0
df = pd.concat([true, fake], ignore_index=True)
# df.drop_duplicates(subset=["title"], keep="last", inplace=True)
df = df.sample(frac=1)
df.reset_index(inplace=True)
len(df)

44898

In [5]:
import re


def clean_text(text):
  text = text.lower().strip()
  text = re.sub(r"https?://\S+|www\.\S+", "", text)
  text = re.sub(r"\[.*?\]", "", text)
  text = re.sub(r"\w*\d\w*", "", text)
  text = re.sub(r"<.*?>+", "", text)
  text = re.sub(r"\n", " ", text)
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r"\W", " ", text)
  return text


In [6]:
df["text"] = df["text"].apply(clean_text)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  df["text"], df["label"], test_size=0.25
)

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5788
           1       1.00      0.99      1.00      5437

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

0.9961692650334075


In [96]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5788
           1       0.99      0.99      0.99      5437

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

0.9889532293986637


In [ ]:
# Good classification models
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(model.score(X_test, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5788
           1       1.00      1.00      1.00      5437

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

0.9964365256124722


In [98]:
# save the model and vectorizer
import pickle

with open("gradient_boosting_classifier.pkl", "wb") as f:
  pickle.dump(model, f)
with open("tfidf_vectorizer.pkl", "wb") as f:
  pickle.dump(vect, f)

In [ ]:
import pickle

with open("gradient_boosting_classifier.pkl", "rb") as f:
  model: GradientBoostingClassifier = pickle.load(f)

with open("tfidf_vectorizer.pkl", "rb") as f:
  vect: TfidfVectorizer = pickle.load(f)

In [ ]:
sample_texts = [
  "Breaking news: Scientists discover a cure for the common cold!",
  "You won't believe what this celebrity did last night!",
  "Trump won the 2020 election by a landslide!",
  "New study shows that eating chocolate improves brain function.",
  "Scientists discover new cure for cancer",
  "Trump lost the 2020 election due to widespread fraud",
  "Joe Biden won the 2020 presidential election",
]

sample_texts = vect.transform(sample_texts)
predictions = model.predict(sample_texts)
print(predictions)

[0 0 0 0 0 0 0]
